In [ ]:
import os
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv2D, concatenate, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, Dropout, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
import tensorflow as tf

TPU_PATH = f"grpc://{os.environ['COLAB_TPU_ADDR']}"

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_PATH)
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.64.26.82:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.64.26.82:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
from PIL import Image
import os, glob, sys
import numpy as np
from sklearn.model_selection import train_test_split # 섞어줌
from keras.utils import np_utils

In [ ]:
X_train, X_test, y_train, y_test = np.load('/content/drive/MyDrive/Colab Notebooks/전일반/day13/categorical_image_data_64.npy',
                                           allow_pickle=True)

X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

print(X_train.shape)       
print(X_train.shape[0])     
print(np.bincount(y_train)) 
print(np.bincount(y_test))  

(9095, 64, 64, 3)
9095
[2800 1472 3138 1685]
[298 196 336 181]


In [ ]:
def model_res():

    def indentity_block(X, f, filters):
        
        F1, F2, F3 = filters
        X_shortcut = X

        X = Conv2D(F1, (1, 1), strides = (1, 1), padding = 'valid')(X)
        X = layers.BatchNormalization(axis = 3)(X)
        X = layers.Activation('relu')(X)

        X = Conv2D(F2, (f, f), strides = (1, 1), padding = 'same')(X)
        X = layers.BatchNormalization(axis = 3)(X)
        X = layers.Activation('relu')(X)

        X = Conv2D(F3, (1, 1), strides = (1, 1), padding = 'valid')(X)
        X = layers.BatchNormalization(axis = 3)(X)

        X = layers.Add()([X, X_shortcut])
        X = layers.Activation('relu')(X)

        return X

    def Conv_Block(X, f, filters, s = 2):
        
        F1, F2, F3 = filters

        X_shortcut = X

        X = Conv2D(F1, (1, 1), strides = (s, s), padding = 'valid')(X)
        X = layers.BatchNormalization(axis = 3)(X)
        X = layers.Activation('relu')(X)

        X = Conv2D(F2, (f, f), strides = (1, 1), padding = 'same')(X)
        X = layers.BatchNormalization(axis = 3)(X)
        X = layers.Activation('relu')(X)

        X = Conv2D(F3, (1, 1), strides = (1, 1), padding = 'valid')(X)
        X = layers.BatchNormalization(axis = 3)(X)


        X_shortcut = Conv2D(F3, (1, 1), strides = (s, s), padding= 'valid')(X_shortcut)   
        X_shortcut = layers.BatchNormalization(axis = 3)(X_shortcut)


        X = layers.Add()([X, X_shortcut])
        X = layers.Activation('relu')(X)

        return X 

    input_img = Input((64, 64, 3))

    X = Conv2D(64, (7, 7), strides = (2, 2), padding = (3, 3))(input_img)
    X = layers.BatchNormalization(axis = 3)(X)
    X = layers.Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides = (2, 2))(X)
    X = Conv_Block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = indentity_block(X, 3, [64, 64, 256])
    X = indentity_block(X, 3, [64, 64, 256])
    X = Conv_Block(X, 3, [128, 128, 512])
    X = indentity_block(X, 3, [128, 128, 512])
    X = indentity_block(X, 3, [128, 128, 512])
    X = Conv_Block(X, 3, [256, 256, 1024])
    X = indentity_block(X, 3, [256, 256, 1024])
    X = indentity_block(X, 3, [256, 256, 1024])
    X = indentity_block(X, 3, [256, 256, 1024])
    X = indentity_block(X, 3, [256, 256, 1024])
    X = indentity_block(X, 3, [256, 256, 1024])
    X = Conv_Block(X, 3, [512, 512, 2048])
    X = indentity_block(X, 3, [512, 512, 2048])
    X = indentity_block(X, 3, [512, 512, 2048])
    X = AveragePooling2D(2, 2)(X)
    X = Flatten()(X)
    out = Dense(4, activation="softmax")(X)

    model = Model(inputs = input_img, outputs = out)

    return model

In [ ]:
model_res().summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_16 (InputLayer)          [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 zero_padding2d_15 (ZeroPadding  (None, 70, 70, 3)   0           ['input_16[0][0]']               
 2D)                                                                                              
                                                                                                  
 conv2d_297 (Conv2D)            (None, 32, 32, 64)   9472        ['zero_padding2d_15[0][0]']      
                                                                                                  
 batch_normalization_292 (Batch  (None, 32, 32, 64)  256         ['conv2d_297[0][0]']       

In [ ]:
with strategy.scope():

    early_stopping = EarlyStopping(monitor='val_loss', patience=10) 

    model = model_res()
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    history = model.fit(X_train, y_train, callbacks=[early_stopping], batch_size=50, validation_data = (X_test,y_test), epochs = 100)

Epoch 1/100
182/182 [==============================] - 83s 214ms/step - loss: 2.2917 - accuracy: 0.3536 - val_loss: 1.6794 - val_accuracy: 0.3462
Epoch 2/100
182/182 [==============================] - 8s 43ms/step - loss: 1.4574 - accuracy: 0.4196 - val_loss: 1.3204 - val_accuracy: 0.3927
Epoch 3/100
182/182 [==============================] - 8s 43ms/step - loss: 1.1882 - accuracy: 0.4953 - val_loss: 1.1648 - val_accuracy: 0.5292
Epoch 4/100
182/182 [==============================] - 8s 44ms/step - loss: 1.1089 - accuracy: 0.5402 - val_loss: 9.1982 - val_accuracy: 0.3215
Epoch 5/100
182/182 [==============================] - 8s 44ms/step - loss: 1.0567 - accuracy: 0.5665 - val_loss: 1.0873 - val_accuracy: 0.5440
Epoch 6/100
182/182 [==============================] - 8s 44ms/step - loss: 0.9881 - accuracy: 0.6010 - val_loss: 1.9563 - val_accuracy: 0.5401
Epoch 7/100
182/182 [==============================] - 8s 43ms/step - loss: 0.9410 - accuracy: 0.6305 - val_loss: 3.0372 - val_accurac